In [1]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.frequent_patterns import apriori

## Preprocess Data

In [9]:
data = pd.read_csv('皇室對戰分析/merged.csv')
cards = pd.read_csv('皇室對戰分析/cards.csv')['Name']

cols = ['cards_0_name', 'cards_1_name', 'cards_2_name', 'cards_3_name', 
        'cards_4_name', 'cards_5_name', 'cards_6_name', 'cards_7_name']
TeamDeck = pd.DataFrame(data.iloc[:,6:14]).set_axis(cols, axis = 1)
OpponentDeck = pd.DataFrame(data.iloc[:,14:]).set_axis(cols, axis = 1)
decks = pd.concat([TeamDeck, OpponentDeck], ignore_index=True).drop_duplicates()

data = []
for index, deck in decks.iterrows():
    table = [False for i in range(len(cards))]
    for col in cols:
        card_num = cards[cards == deck[col]].index[0]
        table[card_num] = True
    data.append(table)

data = pd.DataFrame(data, columns=cards)
data

Name,Electro Spirit,Fire Spirit,Heal Spirit,Ice Spirit,Skeletons,Bats,Bomber,Goblins,Spear Goblins,Archers,...,Rage,Clone,Goblin Barrel,Tornado,Freeze,Poison,Lightning,The Log,Graveyard,Party Rocket
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,True,False,False,False,True,False,False,...,False,False,False,True,False,False,False,True,False,False
2,False,False,False,False,False,False,True,False,False,False,...,False,False,False,False,True,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
4,False,False,False,True,False,False,False,False,False,False,...,False,False,True,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9092,False,False,False,True,False,True,False,False,False,False,...,False,False,False,True,False,False,False,True,False,False
9093,False,False,False,False,False,False,False,False,True,False,...,False,False,False,False,False,True,False,False,False,False
9094,False,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,True,False,True,False,False
9095,False,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,True,False,False,False


## 各卡使用率

In [3]:
card_count = pd.concat([cards, pd.DataFrame([0]*len(cards), columns = ['count'])], axis = 1)

"""for deck in decks.values:
    for i in deck:
        x = (card_count['Name'] == i)
        card_count.loc[x] = i, card_count.loc[x]['count'].values[0]+1"""

for card in cards:
    x = decks[(decks.iloc[:, :] == card)]
    card_count.loc[card_count['Name'] == card] = card, x.count().values.sum()

useRate = pd.DataFrame([i / decks.shape[0] for i in card_count['count']], columns = ['UseRate'])
card_count = pd.concat([card_count, useRate], axis = 1)
card_count = card_count.loc[card_count['count'] > 0].sort_values(by = ['count'], ascending=False, ignore_index = True)

print(card_count.shape)
card_count.head()

(109, 3)


,Name,count,UseRate
0,The Log,3526,0.387600
1,Goblins,2368,0.260306
2,Phoenix,2300,0.252831
3,Tornado,2101,0.230955
4,Barbarian Barrel,1992,0.218973


In [4]:
precents = [[i/20, 0] for i in reversed(range(20))]
prev_total = 0
for i in precents:
    x = (card_count[card_count['UseRate'] >= i[0]])
    i[1] = x['UseRate'].count() - prev_total
    prev_total = x['UseRate'].count()
    
for i in precents:
    if i[1] == 0:continue
    print(str(int(i[0]*100)) + ' - ' + str(int(i[0]*100) + 5) + '%', i[1])

35 - 40% 1
25 - 30% 2
20 - 25% 3
15 - 20% 4
10 - 15% 10
5 - 10% 45
0 - 5% 44


## Apriori

In [5]:
apriori(data, min_support=0.07)

,support,itemsets
0,0.171815,(3)
1,0.094757,(4)
2,0.074420,(5)
3,0.260306,(7)
4,0.083984,(11)
...,...,...
60,0.111245,"(107, 103)"
61,0.089700,"(3, 111, 7)"
62,0.072881,"(3, 29, 111)"
63,0.070902,"(3, 77, 111)"


In [6]:
apriori(data, min_support=0.07, use_colnames=True)

,support,itemsets
0,0.171815,(Ice Spirit)
1,0.094757,(Skeletons)
2,0.074420,(Bats)
3,0.260306,(Goblins)
4,0.083984,(Goblin Gang)
...,...,...
60,0.111245,"(Barbarian Barrel, Tornado)"
61,0.089700,"(The Log, Goblins, Ice Spirit)"
62,0.072881,"(The Log, Ice Spirit, Hog Rider)"
63,0.070902,"(The Log, Ice Spirit, Mighty Miner)"


## FP Growth

In [7]:
fpgrowth(data, min_support=0.07)

,support,itemsets
0,0.208530,(97)
1,0.197867,(59)
2,0.085962,(56)
3,0.085962,(42)
4,0.079697,(63)
...,...,...
60,0.082884,"(77, 7)"
61,0.072441,"(3, 77)"
62,0.074310,"(111, 77, 7)"
63,0.070902,"(3, 77, 111)"


In [8]:
fpgrowth(data, min_support=0.07, use_colnames=True)

,support,itemsets
0,0.208530,(Arrows)
1,0.197867,(Miner)
2,0.085962,(Bandit)
3,0.085962,(Dark Prince)
4,0.079697,(Inferno Dragon)
...,...,...
60,0.082884,"(Goblins, Mighty Miner)"
61,0.072441,"(Ice Spirit, Mighty Miner)"
62,0.074310,"(The Log, Goblins, Mighty Miner)"
63,0.070902,"(The Log, Ice Spirit, Mighty Miner)"
